In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

In [3]:
es_client.info()

ObjectApiResponse({'name': '4a1d4f08d8d0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LyyJsMWXS56JFL3RyvY4VQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name = "home"
es_client.indices.create(index= index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'home'})

In [5]:
from tqdm.auto import tqdm

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [9]:
def homework1(query):
    homework_body = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=homework_body)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_score'])
    return result_docs

In [10]:
homework1("How do I execute a command in a running docker container")

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

In [1]:
def homework2(query):
    homework_body = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=homework_body)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit)
    return result_docs

In [13]:
homework2('How do I execute a command in a running docker container?')

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    combined_context = ""
    for doc in search_results:
        context_template = f"Q: {doc['question']}\nA: {doc['text']}\n\n"
        combined_context += context_template
        
    prompt = prompt_template.format(question=query, context=combined_context).strip()
    return prompt

In [26]:
query = "How do I execute a command in a running docker container?"

def rag(query):
    search_results = homework2(query)
    prompt = build_prompt(query, search_results)
    return prompt

In [27]:
len(rag(query))

1487

In [17]:
prompt = rag(query)

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [20]:
encoded_prompt = encoding.encode(prompt)
num_tokens = len(encoded_prompt)

In [21]:
num_tokens

329

In [22]:
from openai import OpenAI

client = OpenAI()

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [24]:
llm(prompt)

'To execute a command in a running docker container, you can follow these steps:\n\n1. Find the container ID or name of the running container by using the `docker ps` command:\n    ```sh\n    docker ps\n    ```\n\n2. Once you have identified the container ID or name, you can execute a command inside the specific container using the `docker exec` command. For example, to start a bash shell in the running container, use:\n    ```sh\n    docker exec -it <container-id> bash\n    ```\n\nReplace `<container-id>` with the actual ID or name of your container.'